In [11]:
import numpy as np
from random import randint
from itertools import pairwise

In [99]:
# Use np.random.permutation
def shuffle(arr):
  return np.random.permutation(len(arr))

In [69]:
# Use np.random.shuffle
def shuffle(arr):
  np.random.shuffle(arr)
  return arr

In [106]:
# Use Sattolo's algorithm 
# https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle#Sattolo's_algorithm
def shuffle(items):
  """Sattolo's algorithm."""

  i = len(items)
  while i > 1:
      i = i - 1
      j = randrange(i)  # 0 <= j <= i-1
      items[j], items[i] = items[i], items[j]
  return items

In [6]:
def shuffle(size):
    """
    Returns a random cycle of given size using Durstenfeld algorithm

    https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle#The_modern_algorithm
    """

    assert size > 1, "size should be more than 1"

    items = list(range(size))

    i = len(items) - 1
    while i > 1:
        k = randint(1, i)
        items[k], items[i] = items[i], items[k]
        i -= 1

    return items

In [107]:
N = 100000
count = 0
shuffled = tuple([1, 2, 3])
permutations = {}

while (count < N):
  shuffled = tuple(shuffle(list(shuffled)))

  if shuffled in permutations: 
    permutations[shuffled] += 1
  else:
    permutations[shuffled] = 1
  

  count += 1

permutations

{(2, 3, 1): 33350, (3, 1, 2): 33275, (1, 2, 3): 33375}

In [108]:
frequencies = list(map(lambda x: x/N ,permutations.values()))
frequencies

[0.3335, 0.33275, 0.33375]

In [109]:
min = np.min(frequencies)
max = np.max(frequencies)
min,max

(0.33275, 0.33375)

In [110]:
# Tests
(max - min) / N <= 2 / np.sqrt(N)

True

In [111]:
max <= 1/len(permutations) + 1/np.sqrt(N)

True

In [112]:
min >= 1/len(permutations) - 1/np.sqrt(N)

True

In [7]:
def tupleSort(tuple):
  if np.min(tuple) == tuple[0]:
    return tuple
  else:
    return (tuple[1], tuple[0])

In [208]:
tupleSort((2,1))

(1, 2)

In [106]:
def getCycleEdges(cycle):
  edges = [*list(pairwise(cycle)), (cycle[-1], cycle[0])]
  return list(map(tupleSort, edges))

In [107]:
getCycleEdges([1,2,3])

[(1, 2), (2, 3), (1, 3)]

In [8]:
def expanderFromCycles(k, N):
  assert k > 1, "To build an expander graph you need at least 2 cycles"
  assert N*(N-1) > 2*k, f"There is not enough room for {k} cycles in a graph of size {N}"
  
  permutations = []
  edges = set()

  for i in range(k):
    while len(edges) != (i+1)*N:
      permutation=shuffle(N)

      permutation_edges = [*list(pairwise(permutation)), (permutation[-1], permutation[0])]
      permutation_edges_sorted = list(map(tupleSort, permutation_edges))
      edges_copy = set([*edges, *permutation_edges_sorted])

      if len(edges_copy) == len(edges) + N:
        permutations.append(permutation)
        edges = set([*edges_copy])
  
  return {"permutations": permutations, "edges":list(edges)}
  


In [109]:
def expanderFromCycles2(k, N):
    assert k > 1, "To build an expander graph you need at least 2 cycles"
    assert N*(N-1) > 2*k, f"There is not enough room for {k} cycles in a graph of size {N}"

    permutations = []
    edges = set()

    while len(edges) != k * N:
        permutations = [shuffle(N) for _ in range(k)]
        edges = set(sum([getCycleEdges(cycle) for cycle in permutations], []))

    return {"permutations": permutations, "edges": list(edges)}

In [292]:
k = 2
N = 7
len(expanderFromCycles(k, N)["edges"]) == k*N

True

In [9]:
import networkx as nx

In [14]:
N = 4333
k = 7
G = nx.Graph()
G.add_nodes_from(range(N))
G.add_edges_from(expanderFromCycles(k, N)["edges"])

In [299]:
nx.is_k_regular(G, 2 * k)

True

In [15]:
nx.diameter(G)

5

### Case N = 4333

- k = 1 -> diam = 2166
- k = 2 -> diam = 10
- k = 3 -> diam = 7
- k = 4 -> diam = 6
- k = 5 -> diam = 6
- k = 6 -> diam = 5
- k = 7 -> diam = 5